**Library import**

In [8]:
# Active l'affichage dynamique sur le notebook
import pandas as pd
import matplotlib.pyplot as plt
import math as m
import matplotlib.animation as animation
import numpy as np
import os
%matplotlib qt 

Imported data

In [3]:
data = pd.read_csv("data/csv/11-05-2024/dynamic_B2_B1_O1_A1_A2_0.csv")
data_n = data.drop(columns=["tracker", "id"])

In [101]:
def add_noise(data, noise_level, frequency):
    """
    Adds noise to the data['X'] and data['Y'] columns.
    
    Parameters:
    data (DataFrame): Input data with 'X' and 'Y' columns.
    noise_level (float): The maximum noise level.
    frequency (int): The frequency of noise application.
    
    Returns:
    DataFrame: Data with added noise.
    """
    noisy_data = data.copy()
    num_points = len(data)
    
    noise_indices = np.random.choice(num_points, size=num_points // frequency, replace=False)
    noise_X = np.random.uniform(0, noise_level, size=num_points // frequency)
    noise_Y = np.random.uniform(0, noise_level, size=num_points // frequency)
    
    noisy_data.loc[noise_indices, 'x'] += noise_X
    noisy_data.loc[noise_indices, 'y'] += noise_Y
    
    return noisy_data
data_n = add_noise(data_n, 2, 10)

In [105]:
# save the data to a csv file
data_n.to_csv("data/csv/noisy_dynamic_B2_B1_O1_A1_A2_0.csv", index=False)

**Exclusive radius sorting**

In [ ]:
def exclusive_radius(data_x,data_y,R=0.556):  # R en mètre
    prec_ecart_x = 0
    prec_ecart_y = 0
    cnt = 0
    for i in range(1,len(data_x)):
        
        if(i>1 and (m.sqrt((data_x[i]-data_x[i-1])**2+(data_y[i]-data_y[i-1])**2) <= R) or cnt >= 2):
            cnt = 0
            prec_ecart_x = data_x[i]-data_x[i-1]
            prec_ecart_y = data_y[i]-data_y[i-1]
            
        if(i>2 and (m.sqrt((data_x[i]-data_x[i-1])**2+(data_y[i]-data_y[i-1])**2) > R and cnt < 3)):
            data_x[i] = data_x[i-1] + prec_ecart_x
            data_y[i] = data_y[i-1] + prec_ecart_y
            cnt += 1

    return

data_n_filtered = data_n.copy()

exclusive_radius(data_n_filtered['x'],data_n_filtered['y'])

**EMA sorting**

In [5]:
def exponential_moving_average(data_1, alpha=0.33):
    ema = [data_1[0]]
    for i in range(1,len(data_1)):
        ema.append(alpha * data_1[i] + (1 - alpha) * ema[-1])
    return ema

# Calculate EMA for the 'X' and 'Y' columns
data_n_filtered['x'] = exponential_moving_average(data_n_filtered['x'])
data_n_filtered['y'] = exponential_moving_average(data_n_filtered['y'])

In [6]:
# Initialize the plot
fig, ax = plt.subplots()
line_raw, = ax.plot([], [], 'ro-', label='Raw Data', alpha=0.5)
line_ema, = ax.plot([], [], 'bo-', label='EMA', alpha=0.5)

# Set plot limits
ax.set_xlim(min(data_n['x']) - 1, max(data_n['y']) + 1)
ax.set_ylim(min(data_n_filtered['x']) - 1, max(data_n_filtered['y']) + 1)

ax.legend()

# Show the real points
A1 = [1, 0]
A2 = [1, 3.3]
B1 = [5.5, 0]
B2 = [5.5, 3.3]
O1 = [3.5, 0]
O2 = [3.5, 0.8]
O3 = [3.5, 1.8]
O4 = [3.5, 3.3]

# Make points always on top
plt.scatter(A1[0], A1[1], c='b', label='A1', marker='x')
plt.scatter(A2[0], A2[1], c='b', label='A2', marker='x')
plt.scatter(B1[0], B1[1], c='b', label='B1', marker='x')
plt.scatter(B2[0], B2[1], c='b', label='B2', marker='x')
plt.scatter(O1[0], O1[1], c='b', label='O1', marker='x')
plt.scatter(O2[0], O2[1], c='b', label='O2', marker='x')
plt.scatter(O3[0], O3[1], c='b', label='O3', marker='x')
plt.scatter(O4[0], O4[1], c='b', label='O4', marker='x')

# Add text annotations
texts = [
    ax.text(A1[0]-0.1, A1[1], 'A1', fontsize=10, ha='right'),
    ax.text(A2[0]-0.1, A2[1], 'A2', fontsize=10, ha='right'),
    ax.text(B1[0]-0.1, B1[1], 'B1', fontsize=10, ha='right'),
    ax.text(B2[0]-0.1, B2[1], 'B2', fontsize=10, ha='right'),
    ax.text(O1[0]-0.1, O1[1], 'O1', fontsize=10, ha='right'),
    ax.text(O2[0]-0.1, O2[1], 'O2', fontsize=10, ha='right'),
    ax.text(O3[0]-0.1, O3[1], 'O3', fontsize=10, ha='right'),
    ax.text(O4[0]-0.1, O4[1], 'O4', fontsize=10, ha='right')
]

# Initialization function
def init():
    line_raw.set_data([], [])
    line_ema.set_data([], [])
    return line_raw, line_ema

# Animation function
def animate(i):
    x_raw = data_n['x'][:i]
    y_raw = data_n['y'][:i]
    x_ema = data_n_filtered['x'][:i]
    y_ema = data_n_filtered['y'][:i]
    line_raw.set_data(x_raw, y_raw)
    line_ema.set_data(x_ema, y_ema)
    for text in texts:
        text.set_zorder(3)  # Ensure text stays on top
    return line_raw, line_ema, *texts

# Create the animation
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=data_n.shape[0], interval=200, blit=True)

# Show the plot with animation
plt.xlim(-1, 8)
plt.ylim(-1, 6)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Raw Data and EMA Plot')
plt.show()

In [7]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Create a figure with two subplots next to each other
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Initialize the plots
line_raw, = ax1.plot([], [], 'ro-', label='Raw Data', alpha=0.5)
line_ema, = ax2.plot([], [], 'bo-', label='EMA', alpha=0.5)

# Set plot limits for both subplots
ax1.set_xlim(min(data_n['x']) - 1, max(data_n['x']) + 1)
ax1.set_ylim(min(data_n['y']) - 1, max(data_n['y']) + 1)
ax2.set_xlim(min(data_n_filtered['x']) - 1, max(data_n_filtered['x']) + 1)
ax2.set_ylim(min(data_n_filtered['y']) - 1, max(data_n_filtered['y']) + 1)

# Set labels and titles for both subplots
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('Raw Data Plot')
ax1.legend()

ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_title('EMA Plot')
ax2.legend()

# Real points
A1 = [1, 0]
A2 = [1, 3.3]
B1 = [5.5, 0]
B2 = [5.5, 3.3]
O1 = [3.5, 0]
O2 = [3.5, 0.8]
O3 = [3.5, 1.8]
O4 = [3.5, 3.3]

# Plot points on both subplots
for ax in [ax1, ax2]:
    ax.scatter(A1[0], A1[1], c='b', marker='x')
    ax.scatter(A2[0], A2[1], c='b', marker='x')
    ax.scatter(B1[0], B1[1], c='b', marker='x')
    ax.scatter(B2[0], B2[1], c='b', marker='x')
    ax.scatter(O1[0], O1[1], c='b', marker='x')
    ax.scatter(O2[0], O2[1], c='b', marker='x')
    ax.scatter(O3[0], O3[1], c='b', marker='x')
    ax.scatter(O4[0], O4[1], c='b', marker='x')

# Text annotations
texts = [
    ax1.text(A1[0]-0.1, A1[1], 'A1', fontsize=10, ha='right'),
    ax1.text(A2[0]-0.1, A2[1], 'A2', fontsize=10, ha='right'),
    ax1.text(B1[0]-0.1, B1[1], 'B1', fontsize=10, ha='right'),
    ax1.text(B2[0]-0.1, B2[1], 'B2', fontsize=10, ha='right'),
    ax1.text(O1[0]-0.1, O1[1], 'O1', fontsize=10, ha='right'),
    ax1.text(O2[0]-0.1, O2[1], 'O2', fontsize=10, ha='right'),
    ax1.text(O3[0]-0.1, O3[1], 'O3', fontsize=10, ha='right'),
    ax1.text(O4[0]-0.1, O4[1], 'O4', fontsize=10, ha='right')
]

# Initialization function
def init():
    line_raw.set_data([], [])
    line_ema.set_data([], [])
    return line_raw, line_ema

# Animation function
def animate(i):
    x_raw = data_n['x'][:i]
    y_raw = data_n['y'][:i]
    x_ema = data_n_filtered['x'][:i]
    y_ema = data_n_filtered['y'][:i]
    line_raw.set_data(x_raw, y_raw)
    line_ema.set_data(x_ema, y_ema)
    for text in texts:
        text.set_zorder(3)  # Ensure text stays on top
    return line_raw, line_ema, *texts

# Create the animation
ani = animation.FuncAnimation(fig, animate, init_func=init, frames=data_n.shape[0], interval=200, blit=True)

# Show the plot with animation
plt.tight_layout()
plt.show()


In [106]:
with open("data_filtered.json", "w") as f:
    f.write(data_n_filtered.to_json(orient='records', lines=True))
    f.write("\n")

with open("data.json", "w") as f:
    f.write(data_n.to_json(orient='records', lines=True))
    f.write("\n")

Save all gifs

In [ ]:
# Read all csv files in the directory with panda
dataframes = []
for filename in os.listdir('data/csv/11-05-2024/'):
    if filename.endswith(".csv"):
        df = pd.read_csv('data/csv/11-05-2024/'+filename)
        # add a column with the filename
        df['filename'] = filename
        dataframes.append(df)

output_dir = 'data/gif/11-05-2024/'
for df in dataframes:
    data_raweee = df.drop(columns=["tracker", "id"])
    data_filtered = data_raweee.copy()
    exclusive_radius(data_filtered['x'],data_filtered['y'])
    data_filtered['x'] = exponential_moving_average(data_filtered['x'])
    data_filtered['y'] = exponential_moving_average(data_filtered['y'])

    # Initialize the plot
    fig, ax = plt.subplots()
    line_raw, = ax.plot([], [], 'ro-', label='Raw Data', alpha=0.5)
    line_ema, = ax.plot([], [], 'bo-', label='EMA', alpha=0.5)

    # Set plot limits
    ax.set_xlim(min(data_raweee['x']) - 1, max(data_raweee['x']) + 1)
    ax.set_ylim(min(data_filtered['y']) - 1, max(data_filtered['y']) + 1)

    ax.legend()

    # Initialization function
    def init():
        line_raw.set_data([], [])
        line_ema.set_data([], [])
        return line_raw, line_ema

    # Animation function
    def animate(i):
        x_raw = data_raweee['x'][:i]
        y_raw = data_raweee['y'][:i]
        x_ema = data_filtered['x'][:i]
        y_ema = data_filtered['y'][:i]
        line_raw.set_data(x_raw, y_raw)
        line_ema.set_data(x_ema, y_ema)
        return line_raw, line_ema

    # Create the animation
    ani = animation.FuncAnimation(fig, animate, init_func=init, frames=data_raweee.shape[0], interval=200, blit=True)

    # Show the plot with animation
    plt.xlim(-1, 8)
    plt.ylim(-1, 6)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Raw Data and EMA Plot')

    plt.close()

    # Save the animation as a GIF
    output_file = output_dir + df['filename'][0].replace('.csv', '.gif')
    ani.save(output_file, writer='pillow', fps=5)